In [1]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.4 MB/s 
     |████████████████████████████████| 182 kB 93.1 MB/s 
     |████████████████████████████████| 7.6 MB 54.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.2 MB/s 
     |████████████████████████████████| 212 kB 75.3 MB/s 
     |████████████████████████████████| 115 kB 86.5 MB/s 
     |████████████████████████████████| 127 kB 71.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [4]:
foldername= "/content/drive/My Drive/nlpproject/"

In [5]:
import numpy as np 
import pandas as pd 
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from datasets import load_metric
import datetime
from torch import nn
from transformers import AutoConfig
from transformers import AutoModel
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from torch.nn.modules.loss import CrossEntropyLoss

In [6]:
class CFG:
    str_now = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
    basic_lr=1e-3
    train = True
    debug = False
    offline = False
    models_path = "bert-base-uncased"
    epochs = 50000
    save_all_models = False
    apex = True
    print_freq = 20
    num_workers = 4
    model = "bert-base-uncased"
    loss_func = 'SmoothL1'
    scheduler = 'cosine'
    batch_scheduler = True
    num_cycles = 0.5
    num_warmup_steps = 0
    encoder_lr = 2e-5
    decoder_lr = 2e-5
    min_lr = 1e-6
    llrd = True
    layerwise_lr = 5e-5
    layerwise_lr_decay = 0.9
    layerwise_weight_decay = 0.01
    layerwise_adam_epsilon = 1e-6
    layerwise_use_bertadam = False
    #pooling
    pooling = 'mean' # mean, max, min, attention, weightedlayer
    layer_start = 4
    #init_weight
    init_weight = 'normal' # normal, xavier_uniform, xavier_normal, kaiming_uniform, kaiming_normal, orthogonal
    #re-init
    reinit = True
    reinit_n = 1
    #adversarial
    fgm = False
    awp = False
    adv_lr = 1
    adv_eps = 0.2
    unscale = False
    eps = 1e-6
    betas = (0.9, 0.999)
    max_len = 512
    weight_decay = 0.01
    gradient_accumulation_steps = 1
    max_grad_norm = 1000
    target_cols = ['EI', 'SN', 'TF', 'JP']
    seed = 42
    cv_seed = 42
    n_fold = 4
    trn_fold = list(range(n_fold))
    batch_size = 64
    n_targets = 4
    gpu_id = 0
    device = f'cuda:{gpu_id}'
    hidden_size = 768
    init_weight="normal"
cfg=CFG()

In [7]:
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained(cfg.model)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [8]:
#path=foldername+"dataset2(sep).csv"
#print(dataset[0])
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
def getdl(ds):
    total_len=len(ds)
    train_len=int(len(ds)*0.9)
    val_len=int((total_len-train_len)/2)
    test_len=total_len-train_len-val_len
    [train_ds, val_ds, test_ds]=torch.utils.data.random_split(ds, [train_len, val_len, test_len])
    #return (training dataloader, validation dataloader, test dataloader)
    return len(train_ds), len(val_ds), len(test_ds), DataLoader(train_ds, batch_size=cfg.batch_size, shuffle=False, collate_fn=data_collator), DataLoader(val_ds, batch_size=cfg.batch_size, shuffle=False, collate_fn=data_collator), DataLoader(test_ds, batch_size=cfg.batch_size, shuffle=False, collate_fn=data_collator)
    #return DataLoader(ds, batch_size=cfg.batch_size, shuffle=False, collate_fn=data_collator)

In [9]:
#On GPU
input_ids=torch.load(foldername+'Extraced_feature_tensors.pt')
labels=torch.load(foldername+'Extraced_label_tensors.pt')
#On CPU
#input_ids=torch.load(foldername+'Extraced_feature_tensors.pt', map_location=torch.device('cpu'))
#labels=torch.load(foldername+'Extraced_label_tensors.pt', map_location=torch.device('cpu'))

In [128]:
num_one=torch.count_nonzero(labels, dim=0)
num_zero=len(labels)-num_one

print(num_zero)
print(num_one)
LABEL_RATIO=torch.tensor([num_one/num_one, num_zero/num_one])
LABEL_RATIO

tensor(6675)
tensor(1999)


tensor([1.0000, 3.3392])

In [130]:
from torch.utils.data import TensorDataset
a=torch.tensor([1])
b=torch.tensor([0])
class Extracted_ds(Dataset):
    def __init__(self, input_ids, labels):
        self.features=input_ids
        self.labels=labels
    def __len__(self):
        return (len(self.labels))
    def __getitem__(self, index):
        input_id=self.features[index]
        if self.labels[index]==1:
          label=1
        else:
          label=0
        return {"input_ids": input_id, "labels":label}

In [131]:
ds=Extracted_ds(input_ids, labels)

In [133]:
len_train, len_val, len_test, train_dl_EI,val_dl_EI, test_dl_EI=getdl(ds)

In [134]:
class myModel(nn.Module):
    def __init__(self, CFG, dropoutrate):
        super().__init__()
        self.CFG = CFG
        self.relu = nn.ReLU()
        self.t = nn.Tanh()
        self.fc1 = nn.Linear(768, 768)
        self.fc2 = nn.Linear(768, 768)
        self.fc3 = nn.Linear(768, 512)
        self.fc4 = nn.Linear(512, 512)
        self.fc5 = nn.Linear(512, 256)
        self.fc6 = nn.Linear(256, 128)
        self.fc7 = nn.Linear(128, 64)
        self.fc8 = nn.Linear(64, 32)
        self.fc9 = nn.Linear(32, 16)
        self.fc10 = nn.Linear(16, 2)
        self.dropout = nn.Dropout(p=dropoutrate, inplace=False)
        self.sig = nn.Sigmoid()
    
    def forward(self, input_ids):
        output = self.dropout(input_ids)

        output = self.fc1(output)
        output = self.t(output)
        output = self.dropout(output)

        output = self.fc2(output)
        output = self.t(output)
        output = self.dropout(output)

        output = self.fc3(output)
        output = self.t(output)
        output = self.dropout(output)

        output = self.fc4(output)
        output = self.t(output)
        output = self.dropout(output)

        output = self.fc5(output)
        output = self.t(output)
        output = self.dropout(output)

        output = self.fc6(output)
        output = self.t(output)
        output = self.dropout(output)

        output = self.fc7(output)
        output = self.t(output)
        output = self.dropout(output)

        output = self.fc8(output)
        output = self.t(output)
        output = self.dropout(output)

        output = self.fc9(output)
        output = self.t(output)
        output = self.dropout(output)

        output = self.fc10(output)
        #output = self.sig(output)

        #output = torch.squeeze(output)
        return output #2 number between 0 and 1

In [135]:
def evaluate(labels, outputs):
  answers=(torch.argmax(outputs, dim=1))
  allcorrect=torch.sum(answers==labels)
  return allcorrect

In [136]:
def loss(labels, outputs):
    batch_loss=((labels)-outputs)**2
    return torch.mean(batch_loss)

In [137]:
from torch.optim import lr_scheduler
from torch import nn
from torch import optim
from torch.optim import Adam
from tqdm.notebook import tqdm
def train(train_ds, eval_ds, model, epochs, cfg, type, lr, loss=None):
    if torch.cuda.is_available():  
        dev = "cuda:0" 
    else:  
        dev = "cpu" 
    device = torch.device(dev)
    model = model.to(device)

    #weights=torch.tensor([1., 3.]).cuda()
    #criterion = nn.MSELoss()
    criterion = CrossEntropyLoss(weight=LABEL_RATIO)

    criterion.to(device)
    #criterion = loss
    
    optimizer = Adam(model.parameters(), lr=lr)
    #optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=500, eta_min=1e-15)
    #scheduler= optim.lr_scheduler.ExponentialLR(optimizer, 0.99, last_epoch=- 1, verbose=False)
        
    totalevalloss=0
    totalcorrect=0
    totaldata=0
    with torch.no_grad():
        model.eval()
        for batch in eval_ds:
            batch.to(device)
            blabels=batch["labels"]
            outputs=model(input_ids=batch["input_ids"])
            eloss=criterion(outputs, blabels).item()
            totalevalloss+=eloss
            totalcorrect+=evaluate(blabels, outputs)
            totaldata+=len(blabels)
    totalcorrect_rate=(totalcorrect/(totaldata))
    print("probability that our prediction of ", type, " is correct: ", totalcorrect_rate)
    #print(f'Initial Val Loss: {totalevalloss / len(eval_ds): .3f} ' ) 
    print(f'Initial Val Loss: {totalevalloss / len(eval_ds): .3f} | current lr: {scheduler.get_last_lr()}' ) 
    
    for e in range(epochs):
        totaltrainloss=0
        totaltraincorrect=0
        totaltraindata=0
        for i,batch in enumerate(train_ds):
            model.train()
            optimizer.zero_grad()
            batch.to(device)
            labels=batch["labels"]
            outputs=model(input_ids=batch["input_ids"])
            bloss=criterion(outputs, labels)
            bloss.backward()
            optimizer.step()
            totaltrainloss+=bloss.item()
            totaltraincorrect+=evaluate(labels, outputs)
            totaltraindata+=len(labels)
        scheduler.step()
        totalevalloss=0
        totalcorrect=0
        totaldata=0
        with torch.no_grad():
            model.eval()
            for batch in eval_ds:
                batch.to(device)
                blabels=batch["labels"]
                outputs=model(input_ids=batch["input_ids"])
                eloss=criterion(outputs, blabels).item()
                totalevalloss+=eloss
                totalcorrect+=evaluate(blabels, outputs)
                totaldata+=len(blabels)
        totalcorrect_rate=(totalcorrect/(totaldata))
        totaltraincorrect_rate = (totaltraincorrect/(totaltraindata))
        print("probability that our prediction of ", type, " is correct: ", totalcorrect_rate)
        print("probability that our prediction of ", type, " is correct in training dataset: ", totaltraincorrect_rate)
        #print(f'Epoch: {e+ 1} | Train Loss: {totaltrainloss / len(train_ds): .8f} | Val Loss: {totalevalloss / len(eval_ds): .3f}' ) 
        print(f'Epoch: {e+ 1} | Train Loss: {totaltrainloss / len(train_ds): .8f} | Val Loss: {totalevalloss / len(eval_ds): .8f} | current lr: {scheduler.get_last_lr()}' ) 

In [143]:
model=myModel(cfg, 0.05)
train(train_dl_EI, val_dl_EI, model, epochs=cfg.epochs, cfg=cfg, type="EI", lr=1e-4, loss=loss)

probability that our prediction of  EI  is correct:  tensor(0.8088, device='cuda:0')
Initial Val Loss:  0.685 | current lr: [0.0001]
probability that our prediction of  EI  is correct:  tensor(0.4516, device='cuda:0')
probability that our prediction of  EI  is correct in training dataset:  tensor(0.6162, device='cuda:0')
Epoch: 1 | Train Loss:  0.68831385 | Val Loss:  0.679 | current lr: [9.999901304280687e-05]
probability that our prediction of  EI  is correct:  tensor(0.5691, device='cuda:0')
probability that our prediction of  EI  is correct in training dataset:  tensor(0.5746, device='cuda:0')
Epoch: 2 | Train Loss:  0.66046381 | Val Loss:  0.666 | current lr: [9.999605221019085e-05]
probability that our prediction of  EI  is correct:  tensor(0.5945, device='cuda:0')
probability that our prediction of  EI  is correct in training dataset:  tensor(0.5857, device='cuda:0')
Epoch: 3 | Train Loss:  0.65115246 | Val Loss:  0.659 | current lr: [9.999111761904054e-05]
probability that our 

KeyboardInterrupt: ignored